# <a href="https://www.pythontutorial.net/advanced-python/python-garbage-collection/" style="color:Tomato">Python Garbage Collection</a>

Ở bài này, ta sẽ tìm hiểu cách mà *Python garbage collection* (Bộ dọn rác Python) hoạt động và cách để tương tác với *garbage collector* (bộ dọn rác) bằng code.

> *Lưu ý:* Cái kernel của jupyter hoạt động hơi ảo, bộ đếm tham chiếu hoạt động nhiều khi không như mình mong đợi. Nên phần code phía dưới có thể copy ra chỗ khác để chạy thử. Kết quả vẫn sẽ giống như giải thích ở bài giảng.

### Tables of Contents
* [Introduction to Python garbage collection](#1)
* [Interacting with Python garbage collector](#2)
* [Summary](#sum)

## <a class="anchor" id="1">Introduction to Python garbage collection</a>

Trong C hoặc C++, bạn hoàn toàn chịu trách nhiệm cho việc quản lý bộ nhớ cho chương trình của mình.

Trong Python, <span style="color:DarkOrange">bạn không cần phải lo quản lý bộ nhớ vì Python sẽ làm việc đó một cách tự động cho bạn</span>.

Ở bài trước, ta đã biết Python Memory Manager luôn theo dõi các tham chiếu tới các object trong bộ nhớ. Khi một object có số tham chiếu bằng 0, Memory Manager sẽ tiêu diệt nó để giải phóng bộ nhớ.

Tuy nhiên, không phải lúc nào bộ đếm tham chiếu cũng hoạt động đúng. Chẳng hạn khi một object tham chiếu tới chính nó hoặc có 2 objects tham chiếu vào nhau. Hiện tượng này được gọi là *circular references* (tham chiếu vòng tròn).

Khi Python Memory Manager không giải quyết được vấn đề này, nó sẽ gây ra *memory leak*.

Đây chính là lý do mình cần đến Garbage Collection, để giải quyết vấn đề circular references.

Python cho phép bạn tương tác với Garbage Collection thông qua module `gc`.

## <a class="anchor" id="2">Interacting with Python garbage collector</a>

Ta sẽ thử tạo một circle reference giữa 2 instances của 2 class `A` và `B`. Sau đó ta sẽ sử dụng Garbage Collector để dọn 2 objects này.

Đầu tiên, import thư viện `gc` và `ctypes` và định nghĩa 2 hàm để đếm tham chiếu và kiểm tra xem một object có đang tồn tại trong bộ nhớ hay không.

In [1]:
import gc
import ctypes


def ref_count(address):
    return ctypes.c_long.from_address(address).value


def object_exists(object_id):
    for object in gc.get_objects():
        if id(object) == object_id:
            return True

    return False

Tiếp theo, ta tạo 2 class `A` và `B` tham chiếu tới nhau.

In [2]:
class A:
    def __init__(self):
        self.b = B(self)
        print(f'A: {hex(id(self))}, B: {hex(id(self.b))}')


class B:
    def __init__(self, a):
        self.a = a
        print(f'B: {hex(id(self))}, A: {hex(id(self.a))}')


Ta sẽ thử vô hiệu hoá Garbage Collection bằng cách gọi hàm `disable()`

In [3]:
gc.disable()

Giờ ta sẽ tạo một instance của class `A`, việc này cũng sẽ tự tạo một instance của class `B`.

In [4]:
a = A()

B: 0x1063e20e0, A: 0x1063e1ae0
A: 0x1063e1ae0, B: 0x1063e20e0


Tiếp theo, ta sẽ khởi tạo 2 biến đến lưu địa chỉ của 2 instances trên.

In [5]:
a_id = id(a)
b_id = id(a.b)

Thử đếm xem số tham chiếu của chúng

In [6]:
print(ref_count(a_id))
print(ref_count(b_id))

2
1


Ở đây, có 2 tham chiếu tới instance của class `A` là biến `a` và instance của class `B`.

Kiểm tra xem cả 2 instance có đều đang ở trong bộ nhớ hay không. Tất nhiên là cả 2 đều có.

In [7]:
print(object_exists(a_id))
print(object_exists(b_id))

True
True


Giờ ta sẽ cho biến `a` thành `None`. Lúc này thì instance của `A` chỉ còn đúng một tham chiếu là từ instance của `B`.

In [8]:
a = None

print(ref_count(a_id))
print(ref_count(b_id))

1
1


Lúc này thì đếm tham chiếu của cả 2 instances của `A` và `B` đều là 1.

Ta lại kiểm tra xem cả 2 instances đều có đang tồn tại trong bộ nhớ hay không

In [9]:
print(object_exists(a_id))
print(object_exists(b_id))

True
True


Quả nhiên là vẫn có. Giờ ta sẽ sử dụng Garbage Collector.

In [10]:
gc.collect()

4

<span style="color:DarkOrange">Garbage Collector có khả năng phát hiện ra circular reference. Nó sẽ xoá các objects đó để giải phóng bộ nhớ.</span>

Giờ ta thử kiểm tra xem 2 instances phía trên có còn tồn tại trong bộ nhớ nữa hay không.

In [26]:
print(object_exists(a_id))
print(object_exists(b_id))

False
False


Cả 2 đều không còn trong bộ nhớ nữa, do Garbage Collector đã dọn dẹp chúng.

Ta cũng sẽ kiểm tra luôn số tham chiếu của cả 2 instances xem:

In [43]:
print(ref_count(a_id))  # 0
print(ref_count(b_id))  # 0

0
0


Ra 0 hết là đúng.

## <a class="anchor" id="sum" style="color:Violet"> Tổng kết </a>

- Python quản lý bộ nhớ một cách tự động bằng cách sử dụng bộ đếm tham chiếu Reference Counting và bộ dọn rác Garbage Collector.
- Python Garbage Collector giúp tránh memory leaks bằng cách tự động phát hiện circular references và xoá các objects đó.
- Không bao giờ vô hiệu hoá Garbage Collector trừ khi có lý do, và nếu làm thế thì phải kiểm soát được.